In [3]:
import requests
import pandas as pd
import datetime
import seaborn as sns
sns.set_theme()
sns.set_palette("Paired")
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib notebook
from scipy.ndimage import gaussian_filter1d
from IPython.display import display,Image,HTML
from ipywidgets import interactive
import ipywidgets as widgets
import numpy as np

Auf dieser Seite werden die Daten von https://offenedaten.aachen.de/ verwendet.
Die Daten werden per Kommune angegeben, und auch für die gesamte Städteregion. Die Kommunen geben erst seit dem fünten August gesonderte Daten an, zuvor waren die Daten nur für die gesamte Städteregion verfügbar. Dementsprechend beginnen die Graphen über die einzelnen Kommunen zu diesem Zeitpunkt. 

In [4]:
# Getting data, calculating, smoothing, etc

url = 'https://services-eu1.arcgis.com/2ypUQspLVcN0KOBE/arcgis/rest/services/CoronavirusFallzahlen_%C3%B6ffentlich/FeatureServer/1/query?where=1%3D1&outFields=Kommune,Meldedatum,Positiv,Genesen,Tote,Aktiv,Inzidenz,Creator&outSR=4326&f=json'
data = requests.get(url).json()
data=data["features"]

data_table=pd.DataFrame.from_dict({i:z for i,x in enumerate(data) for y,z in x.items()},orient='index').drop(["Creator"],axis=1)
data_table["Meldedatum"] = data_table["Meldedatum"].apply(lambda d: datetime.datetime.fromtimestamp(int(d)/1000).strftime('%Y-%m-%d %H:%M:%S'))
data_table["Meldedatum"] = pd.to_datetime(data_table["Meldedatum"])
data_table["Tote_diff"] = data_table.groupby(['Kommune'])['Tote'].diff()
data_table["Tote_diff"].loc[data_table['Tote_diff'] < 1 ] = np.nan
data_table["Neuinfektionen"] = data_table.groupby(['Kommune'])["Positiv"].diff()

data_table["Neuinfektionen_7dm"] = data_table["Neuinfektionen"].rolling(7).mean()
data_table["Inzidenz_smoothed"] = gaussian_filter1d(data_table.Inzidenz,3)
data_table["last_7"] = data_table["Neuinfektionen"].rolling(7).sum()
data_table["last_4"] = data_table["Neuinfektionen"].rolling(4).sum()
data_table["r4"] = data_table["last_4"]/data_table["last_4"].shift(4)
data_table["r7"] = data_table["last_7"]/data_table["last_7"].shift(7)

data_table.set_index("Meldedatum",inplace=True)

/home/ulli/snap/jupyter/common/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
STATISTIK_SPEC={
    "Positiv":
    {"desc":"Kumulierte Anzahl Erkrankter",
    "plot_type":"lineplot",
    "sharey":{"Genesen","Tote","Positiv"},
    "unit":"Anzahl Fälle"},
    "Genesen":
    {"desc":"Kumulierte Anzahl Genesener",
    "plot_type":"lineplot",    
     "sharey":{"Genesen","Tote","Positiv"},
    "unit":"Anzahl Fälle"},
    "Tote":
    {"desc":"Kumulierte Anzahl Verstorbener",
    "plot_type":"lineplot",    
     "sharey":{"Genesen","Tote","Positiv"},
    "unit":"Anzahl Fälle"}, 
    "Aktiv":
    {"desc":"Momentan Aktive Erkrankungen",
    "plot_type":"lineplot",    
     "sharey":{"Aktiv"},
    "unit":"Anzahl Fälle"}, 
    "Inzidenz":
    {"desc":"Siebentageinzidenz",
    "plot_type":"lineplot",    
     "sharey":{"Inzidenz","Inzidenz_smoothed"},
    "unit":"Fälle pro 100 000 Einwohner"}, 
    "Tote_diff":
    {"desc":"Verstorbene pro Tag",
    "plot_type":"scatterplot",    
     "sharey":{"Tote_diff"},
    "unit":"Anzahl Fälle pro Tag"}, 
    "Neuinfektionen":
    {"desc":"Neuinfektionen pro Tag",
    "plot_type":"lineplot",    
     "sharey":{"Neuinfektionen","Neuinfektionen_7dm"},
    "unit":"Anzahl Fälle pro Tag"}, 
    "Neuinfektionen_7dm":
    {"desc":"Neuinfektionen sieben Tage Mittel",
    "plot_type":"lineplot",    
     "sharey":{"Neuinfektionen","Neuinfektionen_7dm"},
    "unit":"Anzahl Fälle/7 Tage"}, 
    "Inzidenz_smoothed":
    {"desc":"Siebentageinzidenz geglättet",
    "plot_type":"lineplot",    
     "sharey":{"Inzidenz","Inzidenz_smoothed"},
    "unit":"Fälle pro 100 000 Einwohner"},
    "r4":
    {"desc":"Vier-Tage Reproduktionszahl",
    "plot_type":"lineplot",    
     "sharey":{"r4","r7"},
    "unit": "Reproduktionszahl"}, 
    "r7":
    {"desc":"Sieben-Tage Reproduktionszahl",
    "plot_type":"lineplot",    
     "sharey":{"r4","r7"},
    "unit": "Reproduktionszahl"}
}

In [6]:
def zeigekommune(kommune,statistik):
    fig, g = plt.subplots(sharex=True,sharey=True)
    if STATISTIK_SPEC[statistik]["plot_type"]=="lineplot":
        g = sns.lineplot(x="Meldedatum",y=statistik,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
    elif STATISTIK_SPEC[statistik]["plot_type"]=="scatterplot":
        g = sns.scatterplot(x="Meldedatum",y=statistik,color="r", legend="full",
            data=data_table.loc[data_table.Kommune == kommune], ax=g)

    g.yaxis.grid(True)
    g.set(ylabel=STATISTIK_SPEC[statistik]["desc"], xlabel='Meldedatum')
    plt.xticks(rotation=45)
    g.legend([kommune])
    plt.show()

In [7]:
u = interactive(zeigekommune, kommune=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Städteregion",
    rows=11,
    description='Kommune:',
),statistik=widgets.Dropdown(
    options=[(STATISTIK_SPEC[x]["desc"],x) for x in STATISTIK_SPEC],
    value="Inzidenz_smoothed",
    description='Statistik:',
    rows=len(STATISTIK_SPEC),
));
display(u)

interactive(children=(Dropdown(description='Kommune:', index=8, options=('Aachen', 'Alsdorf', 'Baesweiler', 'E…

In [8]:
def vergleichekommunen(kommune1,kommune2,statistik):
    fig, g = plt.subplots(sharex=True,sharey=True)
    if STATISTIK_SPEC[statistik]["plot_type"]=="lineplot":
        g = sns.lineplot(x="Meldedatum",y=statistik,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune1], ax=g)
        sns.lineplot(x="Meldedatum",y=statistik, color="b", legend="full",
                data=data_table.loc[data_table.Kommune == kommune2], ax=g)
    elif STATISTIK_SPEC[statistik]["plot_type"]=="scatterplot":
        g = sns.scatterplot(x="Meldedatum",y=statistik,color="r", legend="full",
            data=data_table.loc[data_table.Kommune == kommune1], ax=g)
        sns.scatterplot(x="Meldedatum", y=statistik, color="b", legend="full",
            data=data_table.loc[data_table.Kommune == kommune2], ax=g)

    g.yaxis.grid(True)
    g.set(ylabel=STATISTIK_SPEC[statistik]["desc"], xlabel='Meldedatum')
    g.legend([kommune1,kommune2])
    plt.xticks(rotation=45)
    plt.show()

In [9]:
w = interactive(vergleichekommunen, kommune1=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Aachen",
    rows=11,
    description='Kommune 1:',
),kommune2=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Alsdorf",
    rows=11,
    description='Kommune 2:',
),statistik=widgets.Dropdown(
    options=[(STATISTIK_SPEC[x]["desc"],x) for x in STATISTIK_SPEC],
    value="Inzidenz_smoothed",
    description='Statistik 2:',
    rows=len(STATISTIK_SPEC),
)
);
display(w)

interactive(children=(Dropdown(description='Kommune 1:', options=('Aachen', 'Alsdorf', 'Baesweiler', 'Eschweil…

In [10]:
def vergleichestatistik(kommune,statistik1,statistik2):
    if statistik2 in STATISTIK_SPEC[statistik1]["sharey"]:
        same_y=True
    else:
        same_y=False

    if same_y:
        fig, g = plt.subplots(sharex=True,sharey=same_y)    
        if STATISTIK_SPEC[statistik1]["plot_type"]=="lineplot":
            g = sns.lineplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
            sns.lineplot(x="Meldedatum",y=statistik2,color="b", legend="full",
            data=data_table.loc[data_table.Kommune == kommune], ax=g)
        elif STATISTIK_SPEC[statistik1]["plot_type"]=="scatterplot":
            g = sns.scatterplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
            sns.scatterplot(x="Meldedatum",y=statistik2,color="b", legend="full",
            data=data_table.loc[data_table.Kommune == kommune], ax=g)
        g.legend([STATISTIK_SPEC[statistik1]["desc"], STATISTIK_SPEC[statistik2]["desc"]])



    else:
        fig, g = plt.subplots(sharex=True)    
        if STATISTIK_SPEC[statistik1]["plot_type"]=="lineplot":
            g = sns.lineplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
        elif STATISTIK_SPEC[statistik1]["plot_type"]=="scatterplot":
            g = sns.scatterplot(x="Meldedatum",y=statistik1,color="r", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=g)
        h = g.twinx()
        if STATISTIK_SPEC[statistik2]["plot_type"]=="lineplot":
            h = sns.lineplot(x="Meldedatum",y=statistik2,color="b", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=h)
        elif STATISTIK_SPEC[statistik2]["plot_type"]=="scatterplot":
            h = sns.scatterplot(x="Meldedatum",y=statistik2,color="b", legend="full",
                data=data_table.loc[data_table.Kommune == kommune], ax=h)
        h.yaxis.grid(False)
        g.legend([STATISTIK_SPEC[statistik1]["desc"]])
        h.legend([STATISTIK_SPEC[statistik2]["desc"]])
        h.set(ylabel=STATISTIK_SPEC[statistik2]["unit"])


    g.set(ylabel=STATISTIK_SPEC[statistik1]["unit"], xlabel='Meldedatum')
    g.yaxis.grid(True)

    plt.show()

In [11]:
v = interactive(vergleichestatistik, kommune=widgets.Dropdown(
    options=data_table["Kommune"].unique().tolist(),
    value="Aachen",
    rows=11,
    description='Kommune:',
),statistik1=widgets.Dropdown(
    options=[(STATISTIK_SPEC[x]["desc"],x) for x in STATISTIK_SPEC],
    value="Inzidenz",
    rows=len(STATISTIK_SPEC),
    description='Statistik 1:',
),statistik2=widgets.Dropdown(
    options=[(STATISTIK_SPEC[x]["desc"],x) for x in STATISTIK_SPEC],
    value="Inzidenz_smoothed",
    description='Statistik 2:',
    rows=len(STATISTIK_SPEC),
)
);

In [12]:
display(v)

interactive(children=(Dropdown(description='Kommune:', options=('Aachen', 'Alsdorf', 'Baesweiler', 'Eschweiler…